In [ ]:
!pip install PyMuPDF
!python -m spacy download en_core_web_lg
!pip install skillNer

In [7]:
import requests
import fitz  # PyMuPDF
import torch
import requests
import json
import numpy as np
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'fitz'

In [4]:
from google.colab import files

uploaded = files.upload()


Saving xgboost_model.pkl to xgboost_model.pkl


In [5]:
import pickle

with open('xgboost_model.pkl', 'rb') as file:
    xgb_model = pickle.load(file)

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [ ]:
def extract_text(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [ ]:
def extract_resume_skills(text):
    annotations = skill_extractor.annotate(text)     #skill ner func.
    resume_skills = set()
    for match in annotations["results"]["full_matches"]:
        docs = match["doc_node_value"]    # don't loop over string.
        if "," in docs:
            split = docs.split(",")
            for s in split:
                resume_skills.add(s.strip())   # strip each skill.
        else:
            resume_skills.add(docs.strip())    # also handle no , skills.
    return resume_skills


In [ ]:
import requests
import json
def jobs(job_role):
  url = "https://jsearch.p.rapidapi.com/search"

  querystring = {
      "query": job_role,
      "page": "1",
      "num_pages": "1"
  }

  headers = {
      "X-RapidAPI-Key": "8be1cbf96bmsh9ff53d5ff4eece3p151a0ajsna71a14a241b4",
      "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
  }

  response = requests.get(url, headers=headers, params=querystring)
  return response

In [ ]:
def extract_job_skills(response_json, skill_extractor):
    job_skills = []

    for job in response_json.get("data", []):
        job_id = job.get("job_id")
        title = job.get("job_title")
        company = job.get("employer_name")
        highlights = job.get("job_highlights", {})

        text_blob = ""
        for section in ["Qualifications", "Responsibilities", "Benefits"]:   #iteratte over all 3 one by one(res in section).
            for item in highlights.get(section, []):
                text_blob += item + ". "

        # skill extraction via skillner.
        extracted = skill_extractor.annotate(text_blob)
        skills = set()
        for match in extracted["results"]["full_matches"]:
            docs = match["doc_node_value"]
            for skill in docs.split(","):
                skills.add(skill.strip())

        job_skills.append({
            "job_id": job_id,
            "title": title,
            "company": company,
            "skills": list(skills)
        })

    return job_skills


In [6]:
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('all-MiniLM-L6-v2')
resume_emb = sbert.encode(resume_skills)
job_emb = sbert.encode(job_skills)
final_input = np.concatenate([resume_emb, job_emb]).reshape(1, -1)
predicted_score = xgb_model.predict(final_input)


NameError: name 'extract_resume_sbert' is not defined